# Hermes — Run a basic Hartree-Fock energy calculation

See [the tutorial](/Tutorials/hermes_hartree_fock_energy_calculation.ipynb) for an in-depth explanation of how the Hermes Hartree-Fock energy calculation works.

In [ ]:
# |hide
WORK_DIR = '~/qdx/hermes_hartree_fock_quickstart/'
!rm -r $WORK_DIR
!mkdir -p $WORK_DIR
%cd $WORK_DIR

/home/machineer/qdx/hermes_hartree_fock_quickstart


In [ ]:
# Get a pdb to work with - we use the pdb-tools cli here but you can download directly from rcsb.org
!pdb_fetch 1brs | pdb_selchain -A | pdb_delhetatm > 1B39_A_nohet.pdb
!ls

1B39_A_nohet.pdb


In [ ]:
import rush
import json
from pathlib import Path

# Get our client, for calling modules and using the rush API
# - edit in your access token and API URL or set the RUSH_TOKEN and RUSH_URL environment variables
client = rush.build_blocking_provider_with_functions(
    batch_tags=["hermes"],
)

(prepared_protein_qdxf, prepared_protein_pdb) = client.prepare_protein(
    Path(Path.home() / 'qdx' / 'hermes_hartree_fock_quickstart' /  "1B39_A_nohet.pdb")
)

# There may be multiple conformers, so select the first one
(first_conformer,) = client.pick_conformer(prepared_protein_qdxf, 0)

# Fragment the protein so that we can run a quantum energy calculation
(fragmented_protein,) = client.fragment_aa(first_conformer, 1, "All", 30)

# Arguments
HERMES_RESOURCES = {
    "gpus": 1,
    "storage": 100,
    "storage_units": "MB",
    "walltime": 60,
}

fragmented_protein_contents = json.load(
    open(fragmented_protein.download(), "r")
)

# Quantum energy calculation
(hermes_energy,) = client.hermes_energy(
    fragmented_protein,
    {},
    {
        "basis": "STO-3G",
        "aux_basis": "6-31G",
        "method": "RestrictedHF",
    },  # configuration for a fast converging, low accuracy run
    None,
    {
        "cutoffs": {"dimer": 15 },
        "level": "Dimer",
        "reference_fragment": len(
            fragmented_protein_contents["topology"]["fragments"]
        )
        - 1,
        
    },
    None,
    resources=HERMES_RESOURCES,
)

energy = json.load(open(hermes_energy.download(), "r"))
# Let's look at the expanded Hartree Fock energy
energy["expanded_hf_energy"]

2024-03-16 10:15:34,990 - rush - INFO - Argument d3c589e3-d757-4528-8173-67ff5b5b593a is now ModuleInstanceStatus.RESOLVING
2024-03-16 10:16:35,558 - rush - INFO - Argument d3c589e3-d757-4528-8173-67ff5b5b593a is now ModuleInstanceStatus.ADMITTED
2024-03-16 10:16:53,885 - rush - INFO - Argument d3c589e3-d757-4528-8173-67ff5b5b593a is now ModuleInstanceStatus.DISPATCHED
2024-03-16 10:16:54,965 - rush - INFO - Argument d3c589e3-d757-4528-8173-67ff5b5b593a is now ModuleInstanceStatus.AWAITING_UPLOAD
2024-03-16 10:17:21,570 - rush - INFO - Argument c461912c-77b1-4563-a746-c73ae9c92120 is now ModuleInstanceStatus.RESOLVING
2024-03-16 10:17:27,061 - rush - INFO - Argument c461912c-77b1-4563-a746-c73ae9c92120 is now ModuleInstanceStatus.ADMITTED
2024-03-16 10:17:31,419 - rush - INFO - Argument c461912c-77b1-4563-a746-c73ae9c92120 is now ModuleInstanceStatus.DISPATCHED
2024-03-16 10:17:32,570 - rush - INFO - Argument c461912c-77b1-4563-a746-c73ae9c92120 is now ModuleInstanceStatus.RUNNING
2024

0.5644545493014448